In [2]:
import logging
import pickle
import json
import time

from pathlib import Path
import uuid
from timeit import default_timer as timer

from geoloc_imc_2023.cbg import CBG, get_prefix_from_ip
from geoloc_imc_2023.query_api import get_measurements_from_tag

from geoloc_imc_2023.default import (
    PROBES_PATH,
    ANCHORS_PATH,
    HITLIST_PATH,
)

logging.basicConfig(level=logging.INFO)

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

NB_TARGET = 3
NB_VP = 10

In [2]:
# load hitlist
with open(HITLIST_PATH, "rb") as f:
    targets_per_prefix = pickle.load(f)

# load probes
with open(PROBES_PATH, "rb") as f:
    probes = pickle.load(f)

# load anchors
with open(ANCHORS_PATH, "rb") as f:
    anchors = pickle.load(f)

## select target and vps dataset

In [3]:
# select targets and vps from anchors
targets= list(anchors.keys())[:NB_TARGET]
vps = {}
for i, probe in enumerate(probes):
    if i >= NB_VP: break
    vps[probe] = probes[probe]

logging.info(f"nb targets: {len(targets)} : {[target for target in targets]}")
logging.info(f"nb_vps : {len(vps)} {[vp for vp in vps]}")

INFO:root:nb targets: 3 : ['213.225.160.239', '145.220.0.55', '5.28.0.17']
INFO:root:nb_vps : 10 ['45.138.229.91', '77.174.76.85', '82.217.219.124', '83.81.82.33', '193.0.0.78', '87.3.187.45', '77.174.30.45', '93.108.63.51', '86.89.224.211', '216.147.123.181']


## Tier 1: probing each target prefixes

In [4]:
dry_run = False
measurement_uuid = uuid.uuid4()
start_time = time.time()

# measurement configuration for retreival
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "mesurement towards all anchors with all probes as vps",
    "type" : "target prefix",
    "af" : 4,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
measurement_config_path = Path(".") / f"../measurements_metadata/{str(measurement_uuid)}.json"
with open(measurement_config_path, "w") as f:
    json.dump(measurement_config, f)

cbg = CBG(ripe_credentials)

# get target prefixes
target_prefixes = []
for target_addr in targets:
    target_prefix = get_prefix_from_ip(target_addr)
    target_prefixes.append(target_prefix)

# measurement for 3 targets in every target prefixes
measurement_config["start_time"], measurement_config["end_time"] = cbg.prefix_probing(
    target_prefixes= target_prefixes, 
    vps= vps,
    targets_per_prefix= targets_per_prefix,
    tag= measurement_uuid,
    dry_run=dry_run
)

INFO:root:starting measurement for target_prefix='213.225.160.0' with ['213.225.160.254', '213.225.160.14', '213.225.160.159'] with 10 vps
DEBUG:root:starting measurement for target_addr='213.225.160.254' with 5
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): atlas.ripe.net:443
DEBUG:urllib3.connectionpool:https://atlas.ripe.net:443 "POST /api/v2/measurements/?key=b3d3d4fc-724e-4505-befe-1ad16a70dc87 HTTP/1.1" 201 27
DEBUG:root:starting measurement for target_addr='213.225.160.254' with 5
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): atlas.ripe.net:443
DEBUG:urllib3.connectionpool:https://atlas.ripe.net:443 "POST /api/v2/measurements/?key=b3d3d4fc-724e-4505-befe-1ad16a70dc87 HTTP/1.1" 201 27
DEBUG:root:starting measurement for target_addr='213.225.160.14' with 5
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): atlas.ripe.net:443
DEBUG:urllib3.connectionpool:https://atlas.ripe.net:443 "POST /api/v2/measurements/?key=b3d3d4fc-724e-4505-bef

## Retrieve prefix probing results 

In [3]:
measurement_results = get_measurements_from_tag("3f3c373e-f213-4230-8652-bfb852481c0c", ripe_credentials["key"])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): atlas.ripe.net:443
DEBUG:urllib3.connectionpool:https://atlas.ripe.net:443 "GET /api/v2/measurements/tags/3f3c373e-f213-4230-8652-bfb852481c0c/results/?key=b3d3d4fc-724e-4505-befe-1ad16a70dc87 HTTP/1.1" 200 None


In [4]:
# test that everything is alright
logging.info(f"nb measurements retrieved: {len(measurement_results)}")

for i, (target_addr, results) in enumerate(measurement_results.items()):
    print(target_addr, ":", len(results))
    

INFO:root:nb measurements retrieved: 9


213.225.160.254 : 7
213.225.160.14 : 7
213.225.160.159 : 7
145.220.0.28 : 7
145.220.0.1 : 7
145.220.0.29 : 7
5.28.0.6 : 7
5.28.0.13 : 7
5.28.0.14 : 7


In [2]:
# save results
out_file = f"../results/{measurement_config['UUID']}.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)

../results/init_measurement.pickle


## Tier 2: probe each target

In [1]:
ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

# load hitlist
hitlist_file = Path(".") / "../datasets/parsed_hitlist.pickle"
with open(hitlist_file, "rb") as f:
    hitlist_addr = pickle.load(f)

targets = [anchor_addr for anchor_addr in anchors][:NB_TARGET]
vps = [anchor_addr for anchor_addr in anchors][:NB_VP]

logging.info(f"nb targets: {len(targets)}")
logging.info(f"nb_vps : {len(vps)}")

In [ ]:
cbg = CBG(targets, vps, ripe_credentials, hitlist_addr, anchors)

start = timer()
measurement_ids = await cbg.target_probing(targets, vps)
end =timer()

time_elapsed = end - start
print(time_elapsed)

In [ ]:
# save results
out_file = f"../results/target_measurement_ids.pickle"
with open(out_file, "wb") as f:
    pickle.dump(measurement_ids,f)

## Retreive target measurements

In [ ]:
# load hitlist
in_file = Path(".") / "../results/target_measurement_ids.pickle"
with open(in_file, "rb") as f:
    measurement_ids = pickle.load(f)

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

print(len(measurement_ids))

measurement_results = []
for measurement_id in measurement_ids:
    measurement_result = retreive_single_measurement(measurement_id, anchors, ripe_credentials["key"])
    print(f"got results for target: {measurement_result['target_addr']} from {len(measurement_result['results'])} servers")
    measurement_results.append(measurement_result)

In [15]:
# save results
out_file = f"../results/target_measurement.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)

../results/target_measurement.pickle
